In [ ]:
%config Completer.use_jedi = False

In [ ]:
%%capture
!cp -r /kaggle/input/label-assignment-nfl/label_assignment/gmmreg-install /kaggle/working/gmmreg-install
%cd /kaggle/working/gmmreg-install/src
!python setup.py install --user
%cd /kaggle/working
%rm -r /kaggle/working/gmmreg-install

In [ ]:
!pip install -Uqqq wandb

In [ ]:
import os
import sys
import wandb

from tqdm.notebook import tqdm
import pandas as pd

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient() 
personal_key_for_api = user_secrets.get_secret("wandb_api")
! wandb login $personal_key_for_api

In [ ]:
gt_labels = (
    pd.read_csv('../input/nfl-health-and-safety-helmet-assignment/train_labels.csv')
    .query('not isSidelinePlayer')
    .query('frame > 0')
)

In [ ]:
sys.path.append("/kaggle/input/label-assignment-nfl")
from label_assignment.all import *
from label_assignment.helmet_assignment.score import NFLAssignmentScorer

In [ ]:
dl = DataLoader(gt_labels, is_train = True, top22 = False)
videos = [v for v in dl.videos if 'Sideline' in v][:30]

def run_sweep():

    with wandb.init(project = 'Label Assignment') as run:

        labeler_cfg = dict(wandb.config)
        print(labeler_cfg)
        
        cfg = {
            'level' : 3,
            'iters' : [30,20,10],
            'n_grid' : 5,
            'scales': [labeler_cfg['scales_0'], labeler_cfg['scales_1'], labeler_cfg['scales_2']],
            'lambdas': [labeler_cfg['lambdas_0'], labeler_cfg['lambdas_1'], labeler_cfg['lambdas_2']]
        }
        
        submission = []
        for video in tqdm(videos):

            ## Filtering the dataloader to return the data of a single video
            dl.filter_video(video = video)

            ## generating submission for a single video
            video_sub, theta = track2sub(dl, **cfg)
            submission.append(video_sub)

        submission = pd.concat(submission)
        submission.to_csv('submission.csv', index = False)
        
        table = wandb.Table(columns=['video', 'score'])
        for video in tqdm(videos):
            scorer = NFLAssignmentScorer(gt_labels.query(f'video == "{video}.mp4"'))
            score = scorer.score(submission)
            table.add_data(video, score)
            print(f'Score for Video {video:>21}: {score:5.3f}')
        scorer = NFLAssignmentScorer(gt_labels.query(f"video in {[v + '.mp4' for v in videos]}"))
        mean_score = scorer.score(submission)
        wandb.log({'mean_score': mean_score, 'results_by_video': table})

In [ ]:
sweep_config = {
    'method': 'bayes',
    'metric' : {
        'name': 'mean_score',
        'goal': 'maximize'   
        }
    }
sweep_config

In [ ]:
import math
parameters_dict = {
    'scales_0' : {
        'distribution': 'log_uniform',
        'min': math.log(0.01),
        'max': math.log(10),
    },
    'scales_1' : {
        'distribution': 'log_uniform',
        'min': math.log(0.01),
        'max': math.log(10),
    },
    'scales_2' : {
        'distribution': 'log_uniform',
        'min': math.log(0.01),
        'max': math.log(10),
    },
    'lambdas_0' : {
        'distribution': 'log_uniform',
        'min': math.log(0.01),
        'max': math.log(10),
    },
    'lambdas_1' : {
        'distribution': 'log_uniform',
        'min': math.log(0.01),
        'max': math.log(10),
    },
    'lambdas_2' : {
        'distribution': 'log_uniform',
        'min': math.log(0.01),
        'max': math.log(10),
    },
}
sweep_config['parameters'] = parameters_dict

In [ ]:
# sweep_id = wandb.sweep(sweep_config, project='NFL_Helmet')
os.environ['WANDB_ENTITY'] = "coldfir3"
os.environ['WANDB_PROJECT'] = "NFL_Helmet"
sweep_id = 'u1y0mkpc'

# Create sweep with ID: u1y0mkpc
# Sweep URL: https://wandb.ai/coldfir3/NFL_Helmet/sweeps/u1y0mkpc

In [ ]:
wandb.agent(sweep_id, run_sweep, count=25)